In [79]:
from dotenv import load_dotenv
from datetime import datetime
import pandas as pd
import regex as re
import json, os, time, pickle, requests, urllib3, urllib.request, sys
sys.path.append('..//venv//Lib//site-packages')
import facebook
import pytesseract

load_dotenv()

token = os.getenv('token')

RAW_DIR = os.path.join('..//data', 'raw')
PIC_DIR = os.path.join('..//data', 'pictures')
STATE_HREF_PATH = os.path.join(RAW_DIR, 'state_pages.pickle')
FB_GROUP_PATH = os.path.join(RAW_DIR, 'facebook_group.csv')

In [3]:
def make_search_dict():
    
    search_dict = {}
    
    with open(STATE_HREF_PATH, 'rb') as file:

        state_hrefs = pickle.load(file)
    for s in state_hrefs:
        search_dict[s.split('/')[-1].capitalize()] = s
    
    return search_dict

In [92]:
def get_page_picture(group_id):
    
    graph = facebook.GraphAPI(access_token=token, version = 3.1)
    name = re.sub('\s', '_' ,graph.request(f'/{group_id}/')['name'])
    pic = graph.request(f'/{group_id}/picture?type=large')['url']
    now = datetime.now()
    dt_string = re.sub('\/', '-', re.sub(':', r'.', re.sub('\s', '_', now.strftime("%d/%m/%Y %H:%M:%S"))))
    os.makedirs(os.path.join(PIC_DIR, f'{name}'), exist_ok=True)
    urllib.request.urlretrieve(pic, os.path.join(PIC_DIR, f'{name}',f'{dt_string}.png'))
    

In [93]:
def get_all_pictures():
    
    id_list = list(pd.read_csv(FB_GROUP_PATH)['id'].values)
    
    for i, l in enumerate(id_list):
        
        get_page_picture(l)
    

In [94]:
get_all_pictures()